In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
from torch.optim import Adam, SGD
import torchvision
import torchvision.transforms as transforms

import sys, os, math
import argparse

In [2]:
lr=0.01
data='cifar10'
root='./data/'
model='vgg'
model_out='./checkpoint/cifar10_vgg_LeakyReLU.pth'
resume = False

In [3]:
if data == 'cifar10':
    nclass = 10
    img_width = 32
    transform_train = transforms.Compose([
#         transforms.RandomCrop(32, padding=4),
#         transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
    ])
    transform_test = transforms.Compose([
        transforms.ToTensor(),
    ])
    trainset = torchvision.datasets.CIFAR10(root=root, train=True, download=True, transform=transform_train)
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=256, shuffle=True, num_workers=8)
    testset = torchvision.datasets.CIFAR10(root=root, train=False, download=True, transform=transform_test)
    testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False, num_workers=8)
    
elif data == 'cifar100':
    nclass = 100
    img_width = 32
    transform_train = transforms.Compose([
#         transforms.RandomCrop(32, padding=4),
#         transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
    ])
    transform_test = transforms.Compose([
        transforms.ToTensor(),
    ])
    trainset = torchvision.datasets.CIFAR100(root=root, train=True, download=True, transform=transform_train)
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=256, shuffle=True, num_workers=8)
    testset = torchvision.datasets.CIFAR100(root=root, train=False, download=True, transform=transform_test)
    testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False, num_workers=8)
    
elif data == 'stl10':
    nclass = 10
    img_width = 32
    transform_train = transforms.Compose([
#         transforms.RandomCrop(32, padding=4),
#         transforms.RandomHorizontalFlip(),
        transforms.Resize((img_width,img_width)),
        transforms.ToTensor(),
    ])
    transform_test = transforms.Compose([
        transforms.Resize((img_width,img_width)),
        transforms.ToTensor(),
    ])
    trainset = torchvision.datasets.STL10(root=root, split='train', transform=transform_train, target_transform=None, download=True)
    testset = torchvision.datasets.STL10(root=root, split='test', transform=transform_test, target_transform=None, download=True)
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=256, shuffle=True, num_workers=8)
    testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False, num_workers=8)

Files already downloaded and verified
Files already downloaded and verified


In [4]:
if model == 'vgg':
    from models.vgg import VGG_LeakyReLU
    net = nn.DataParallel(VGG_LeakyReLU('VGG16', nclass, img_width=img_width).cuda())
    
net

DataParallel(
  (module): VGG_LeakyReLU(
    (features): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): LeakyReLU(negative_slope=0.01)
      (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): LeakyReLU(negative_slope=0.01)
      (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (9): LeakyReLU(negative_slope=0.01)
      (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (12): LeakyReLU(negative_slope=0.01

In [5]:
if resume:
    print(f'==> Resuming from {model_out}')
    net.load_state_dict(torch.load(model_out))

In [6]:
cudnn.benchmark = True

In [7]:
criterion = nn.CrossEntropyLoss()

In [8]:
def train(epoch):
    print('Epoch: %d' % epoch)
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.cuda(), targets.cuda()
        optimizer.zero_grad()
        outputs, _ = net(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        pred = torch.max(outputs, dim=1)[1]
        correct += torch.sum(pred.eq(targets)).item()
        total += targets.numel()
    print(f'[TRAIN] Acc: {100.*correct/total:.3f}')

In [9]:
def test(epoch):
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.cuda(), targets.cuda()
            outputs, _ = net(inputs)
            loss = criterion(outputs, targets)
            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
        print(f'[TEST] Acc: {100.*correct/total:.3f}')

    # Save checkpoint after each epoch
    torch.save(net.state_dict(), model_out)

In [10]:
if data == 'cifar10':
    epochs = [1]
elif data == 'cifar100':
    epochs = [50, 50, 50, 50]
elif data == 'stl10':
    epochs = [50, 50, 50, 50]

In [11]:
count = 0

In [12]:
for epoch in epochs:
    optimizer = Adam(net.parameters(), lr=lr)
    for _ in range(epoch):
        %time train(count)
        %time test(count)
        count += 1
    lr /= 10

Epoch: 0
[TRAIN] Acc: 16.812
CPU times: user 8.24 s, sys: 4.05 s, total: 12.3 s
Wall time: 14 s
[TEST] Acc: 18.870
CPU times: user 725 ms, sys: 651 ms, total: 1.38 s
Wall time: 1.66 s


In [2]:
lr=0.01
data='stl10'
root='./data/'
model='vgg'
model_out='./checkpoint/stl10_vgg_LeakyReLU.pth'
resume = False

In [12]:
for epoch in epochs:
    optimizer = Adam(net.parameters(), lr=lr)
    for _ in range(epoch):
        train(count)
        test(count)
        count += 1
    lr /= 10

Epoch: 0
[TRAIN] Acc: 14.500
[TEST] Acc: 14.113
Epoch: 1
[TRAIN] Acc: 15.260
[TEST] Acc: 16.850
Epoch: 2
[TRAIN] Acc: 16.280
[TEST] Acc: 17.875
Epoch: 3
[TRAIN] Acc: 18.540
[TEST] Acc: 16.650
Epoch: 4
[TRAIN] Acc: 20.300
[TEST] Acc: 21.538
Epoch: 5
[TRAIN] Acc: 22.300
[TEST] Acc: 13.025
Epoch: 6
[TRAIN] Acc: 23.120
[TEST] Acc: 20.988
Epoch: 7
[TRAIN] Acc: 24.940
[TEST] Acc: 23.688
Epoch: 8
[TRAIN] Acc: 24.980
[TEST] Acc: 25.038
Epoch: 9
[TRAIN] Acc: 26.620
[TEST] Acc: 26.100
Epoch: 10
[TRAIN] Acc: 28.020
[TEST] Acc: 24.575
Epoch: 11
[TRAIN] Acc: 29.120
[TEST] Acc: 28.825
Epoch: 12
[TRAIN] Acc: 30.080
[TEST] Acc: 28.137
Epoch: 13
[TRAIN] Acc: 31.340
[TEST] Acc: 28.762
Epoch: 14
[TRAIN] Acc: 32.960
[TEST] Acc: 24.587
Epoch: 15
[TRAIN] Acc: 34.380
[TEST] Acc: 25.975
Epoch: 16
[TRAIN] Acc: 37.180
[TEST] Acc: 29.087
Epoch: 17
[TRAIN] Acc: 37.600
[TEST] Acc: 33.275
Epoch: 18
[TRAIN] Acc: 40.680
[TEST] Acc: 32.237
Epoch: 19
[TRAIN] Acc: 40.820
[TEST] Acc: 28.550
Epoch: 20
[TRAIN] Acc: 43.460


[TRAIN] Acc: 99.980
[TEST] Acc: 59.050
Epoch: 166
[TRAIN] Acc: 100.000
[TEST] Acc: 58.925
Epoch: 167
[TRAIN] Acc: 100.000
[TEST] Acc: 59.075
Epoch: 168
[TRAIN] Acc: 100.000
[TEST] Acc: 59.000
Epoch: 169
[TRAIN] Acc: 100.000
[TEST] Acc: 59.062
Epoch: 170
[TRAIN] Acc: 100.000
[TEST] Acc: 59.100
Epoch: 171
[TRAIN] Acc: 100.000
[TEST] Acc: 59.025
Epoch: 172
[TRAIN] Acc: 100.000
[TEST] Acc: 59.062
Epoch: 173
[TRAIN] Acc: 100.000
[TEST] Acc: 59.013
Epoch: 174
[TRAIN] Acc: 100.000
[TEST] Acc: 59.075
Epoch: 175
[TRAIN] Acc: 100.000
[TEST] Acc: 59.013
Epoch: 176
[TRAIN] Acc: 100.000
[TEST] Acc: 59.087
Epoch: 177
[TRAIN] Acc: 100.000
[TEST] Acc: 58.925
Epoch: 178
[TRAIN] Acc: 100.000
[TEST] Acc: 58.962
Epoch: 179
[TRAIN] Acc: 100.000
[TEST] Acc: 59.013
Epoch: 180
[TRAIN] Acc: 100.000
[TEST] Acc: 58.962
Epoch: 181
[TRAIN] Acc: 100.000
[TEST] Acc: 58.900
Epoch: 182
[TRAIN] Acc: 100.000
[TEST] Acc: 58.900
Epoch: 183
[TRAIN] Acc: 100.000
[TEST] Acc: 58.663
Epoch: 184
[TRAIN] Acc: 100.000
[TEST] Acc:

In [2]:
lr=0.01
data='cifar100'
root='./data/'
model='vgg'
model_out='./checkpoint/cifar100_vgg_LeakyReLU.pth'
resume = False

In [12]:
for epoch in epochs:
    optimizer = Adam(net.parameters(), lr=lr)
    for _ in range(epoch):
        train(count)
        test(count)
        count += 1
    lr /= 10

Epoch: 0
[TRAIN] Acc: 1.712
[TEST] Acc: 2.210
Epoch: 1
[TRAIN] Acc: 3.990
[TEST] Acc: 4.530
Epoch: 2
[TRAIN] Acc: 6.418
[TEST] Acc: 5.280
Epoch: 3
[TRAIN] Acc: 7.978
[TEST] Acc: 5.680
Epoch: 4
[TRAIN] Acc: 10.204
[TEST] Acc: 9.380
Epoch: 5
[TRAIN] Acc: 14.882
[TEST] Acc: 14.420
Epoch: 6
[TRAIN] Acc: 20.014
[TEST] Acc: 13.750
Epoch: 7
[TRAIN] Acc: 24.686
[TEST] Acc: 21.320
Epoch: 8
[TRAIN] Acc: 28.634
[TEST] Acc: 26.230
Epoch: 9
[TRAIN] Acc: 32.614
[TEST] Acc: 29.050
Epoch: 10
[TRAIN] Acc: 36.670
[TEST] Acc: 29.690
Epoch: 11
[TRAIN] Acc: 40.882
[TEST] Acc: 31.930
Epoch: 12
[TRAIN] Acc: 44.938
[TEST] Acc: 35.040
Epoch: 13
[TRAIN] Acc: 49.128
[TEST] Acc: 35.980
Epoch: 14
[TRAIN] Acc: 52.482
[TEST] Acc: 38.850
Epoch: 15
[TRAIN] Acc: 56.378
[TEST] Acc: 38.580
Epoch: 16
[TRAIN] Acc: 60.116
[TEST] Acc: 40.960
Epoch: 17
[TRAIN] Acc: 63.234
[TEST] Acc: 42.740
Epoch: 18
[TRAIN] Acc: 66.300
[TEST] Acc: 42.810
Epoch: 19
[TRAIN] Acc: 69.516
[TEST] Acc: 42.980
Epoch: 20
[TRAIN] Acc: 73.166
[TEST] Ac

[TEST] Acc: 49.000
Epoch: 167
[TRAIN] Acc: 99.980
[TEST] Acc: 49.170
Epoch: 168
[TRAIN] Acc: 99.976
[TEST] Acc: 48.990
Epoch: 169
[TRAIN] Acc: 99.980
[TEST] Acc: 49.080
Epoch: 170
[TRAIN] Acc: 99.982
[TEST] Acc: 49.050
Epoch: 171
[TRAIN] Acc: 99.982
[TEST] Acc: 49.180
Epoch: 172
[TRAIN] Acc: 99.982
[TEST] Acc: 49.060
Epoch: 173
[TRAIN] Acc: 99.980
[TEST] Acc: 48.990
Epoch: 174
[TRAIN] Acc: 99.982
[TEST] Acc: 49.100
Epoch: 175
[TRAIN] Acc: 99.982
[TEST] Acc: 49.090
Epoch: 176
[TRAIN] Acc: 99.982
[TEST] Acc: 49.170
Epoch: 177
[TRAIN] Acc: 99.984
[TEST] Acc: 49.150
Epoch: 178
[TRAIN] Acc: 99.974
[TEST] Acc: 49.290
Epoch: 179
[TRAIN] Acc: 99.978
[TEST] Acc: 49.140
Epoch: 180
[TRAIN] Acc: 99.974
[TEST] Acc: 49.160
Epoch: 181
[TRAIN] Acc: 99.978
[TEST] Acc: 49.170
Epoch: 182
[TRAIN] Acc: 99.980
[TEST] Acc: 49.110
Epoch: 183
[TRAIN] Acc: 99.976
[TEST] Acc: 49.020
Epoch: 184
[TRAIN] Acc: 99.976
[TEST] Acc: 49.110
Epoch: 185
[TRAIN] Acc: 99.982
[TEST] Acc: 49.160
Epoch: 186
[TRAIN] Acc: 99.984
